In [1]:
import os

In [2]:
%pwd

'/Users/charantejakammari/Desktop/DataScience/research'

In [3]:
os.chdir("../")
%pwd

'/Users/charantejakammari/Desktop/DataScience'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass()
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [14]:
from src.Data_Science.constants import *
from src.Data_Science.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        return data_ingestion_config

In [16]:
import zipfile
import urllib.request
from os.path import getsize
import os
from src.Data_Science import logger

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        should_download = True

        if os.path.exists(self.config.local_data_file):
            try:
                with zipfile.ZipFile(self.config.local_data_file, 'r') as existing_zip:
                    existing_zip.testzip()
                logger.info(f"Valid file already exists of size: {getsize(self.config.local_data_file)}")
                should_download = False
            except zipfile.BadZipFile:
                logger.info("Existing file is not a valid zip. Re-downloading.")
                os.remove(self.config.local_data_file)

        if should_download:
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded!")

    def extract_zip_file(self):
        ""
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        ""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile as exc:
            raise zipfile.BadZipFile("Downloaded file is not a valid zip. Check source_URL.") from exc


SyntaxError: invalid syntax (3770094778.py, line 34)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-26 23:17:47,728: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-12-26 23:17:47,730: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-26 23:17:47,731: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-26 23:17:47,732: INFO: common: created directory at: artifacts]
[2025-12-26 23:17:47,733: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-26 23:17:47,901: INFO: 4018249200: artifacts/data_ingestion/data.zip downloaded!]
